In [1]:
# Clone the entire repo.
!git clone -l -s git://github.com/Idan707/Kaggle_TMDB_Box_Office_Prediction.git cloned-repo
%cd cloned-repo
# !ls

Cloning into 'cloned-repo'...
remote: Enumerating objects: 162, done.
remote: Counting objects: 100% (162/162), done.
remote: Compressing objects: 100% (132/132), done.
remote: Total 162 (delta 88), reused 70 (delta 29), pack-reused 0
Receiving objects: 100% (162/162), 18.01 MiB | 25.79 MiB/s, done.
Resolving deltas: 100% (88/88), done.
/content/cloned-repo


In [2]:
#pip install pandas==0.25
pip install catboost

     |████████████████████████████████| 63.9MB 72kB/s 


# Import & Load

In [0]:
# imports
import pandas as pd
import  numpy as np
import ast
import json
import gensim
import datetime
import ast
import nltk
from gensim.utils import simple_preprocess
from gensim.parsing.preprocessing import STOPWORDS
from nltk.stem import WordNetLemmatizer, SnowballStemmer
from sklearn.preprocessing import MultiLabelBinarizer ,normalize
from sklearn import preprocessing
from sklearn.model_selection import train_test_split, StratifiedKFold
from sklearn.metrics import confusion_matrix, roc_auc_score, classification_report
from sklearn.preprocessing import MinMaxScaler, PolynomialFeatures, StandardScaler
from sklearn.pipeline import Pipeline, FeatureUnion, make_pipeline
import lightgbm as lgb
import catboost as cat
from sklearn.metrics import mean_squared_error, mean_squared_log_error
from sklearn.model_selection import GroupKFold
import xgboost as xgb

pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)
pd.options.display.float_format = '{:.2f}'.format

train = pd.read_csv('/content/cloned-repo/Data/train.csv',
                    parse_dates=['release_date'])
test = pd.read_csv('/content/cloned-repo/Data/test.csv',
                    parse_dates=['release_date'])

# Date Fix
train['release_date'] = train['release_date'].mask(train['release_date'].dt.year > 2017, 
                                                   train['release_date'] - pd.offsets.DateOffset(years=100))
test['release_date'] = test['release_date'].mask(test['release_date'].dt.year > 2017, 
                                                   test['release_date'] - pd.offsets.DateOffset(years=100))

test['revenue'] = 0

mlb = MultiLabelBinarizer()

In [0]:
train.loc[train['id'] == 16,'revenue'] = 192864         
train.loc[train['id'] == 90,'budget'] = 30000000                  
train.loc[train['id'] == 118,'budget'] = 60000000       
train.loc[train['id'] == 149,'budget'] = 18000000       
train.loc[train['id'] == 313,'revenue'] = 12000000       
train.loc[train['id'] == 451,'revenue'] = 12000000      
train.loc[train['id'] == 464,'budget'] = 20000000       
train.loc[train['id'] == 470,'budget'] = 13000000       
train.loc[train['id'] == 513,'budget'] = 930000         
train.loc[train['id'] == 797,'budget'] = 8000000        
train.loc[train['id'] == 819,'budget'] = 90000000       
train.loc[train['id'] == 850,'budget'] = 90000000       
train.loc[train['id'] == 1007,'budget'] = 2              
train.loc[train['id'] == 1112,'budget'] = 7500000       
train.loc[train['id'] == 1131,'budget'] = 4300000        
train.loc[train['id'] == 1359,'budget'] = 10000000       
train.loc[train['id'] == 1542,'budget'] = 1             
train.loc[train['id'] == 1570,'budget'] = 15800000       
train.loc[train['id'] == 1571,'budget'] = 4000000        
train.loc[train['id'] == 1714,'budget'] = 46000000       
train.loc[train['id'] == 1721,'budget'] = 17500000       
train.loc[train['id'] == 1865,'revenue'] = 25000000      
train.loc[train['id'] == 1885,'budget'] = 12             
train.loc[train['id'] == 2091,'budget'] = 10             
train.loc[train['id'] == 2268,'budget'] = 17500000       
train.loc[train['id'] == 2491,'budget'] = 6              
train.loc[train['id'] == 2602,'budget'] = 31000000       
train.loc[train['id'] == 2612,'budget'] = 15000000       
train.loc[train['id'] == 2696,'budget'] = 10000000      
train.loc[train['id'] == 2801,'budget'] = 10000000       
train.loc[train['id'] == 335,'budget'] = 2 
train.loc[train['id'] == 348,'budget'] = 12
train.loc[train['id'] == 470,'budget'] = 13000000 
train.loc[train['id'] == 513,'budget'] = 1100000
train.loc[train['id'] == 640,'budget'] = 6 
train.loc[train['id'] == 696,'budget'] = 1
train.loc[train['id'] == 797,'budget'] = 8000000 
train.loc[train['id'] == 850,'budget'] = 1500000
train.loc[train['id'] == 1199,'budget'] = 5 
train.loc[train['id'] == 1282,'budget'] = 9              
train.loc[train['id'] == 1347,'budget'] = 1
train.loc[train['id'] == 1755,'budget'] = 2
train.loc[train['id'] == 1801,'budget'] = 5
train.loc[train['id'] == 1918,'budget'] = 592 
train.loc[train['id'] == 2033,'budget'] = 4
train.loc[train['id'] == 2118,'budget'] = 344 
train.loc[train['id'] == 2252,'budget'] = 130
train.loc[train['id'] == 2256,'budget'] = 1 
train.loc[train['id'] == 2696,'budget'] = 10000000
test.loc[test['id'] == 3033,'budget'] = 250 
test.loc[test['id'] == 3051,'budget'] = 50
test.loc[test['id'] == 3084,'budget'] = 337
test.loc[test['id'] == 3224,'budget'] = 4  
test.loc[test['id'] == 3594,'budget'] = 25  
test.loc[test['id'] == 3619,'budget'] = 500  
test.loc[test['id'] == 3831,'budget'] = 3  
test.loc[test['id'] == 3935,'budget'] = 500  
test.loc[test['id'] == 4049,'budget'] = 995946 
test.loc[test['id'] == 4424,'budget'] = 3  
test.loc[test['id'] == 4460,'budget'] = 8  
test.loc[test['id'] == 4555,'budget'] = 1200000 
test.loc[test['id'] == 4624,'budget'] = 30 
test.loc[test['id'] == 4645,'budget'] = 500 
test.loc[test['id'] == 4709,'budget'] = 450 
test.loc[test['id'] == 4839,'budget'] = 7
test.loc[test['id'] == 3125,'budget'] = 25 
test.loc[test['id'] == 3142,'budget'] = 1
test.loc[test['id'] == 3201,'budget'] = 450
test.loc[test['id'] == 3222,'budget'] = 6
test.loc[test['id'] == 3545,'budget'] = 38
test.loc[test['id'] == 3670,'budget'] = 18
test.loc[test['id'] == 3792,'budget'] = 19
test.loc[test['id'] == 3881,'budget'] = 7
test.loc[test['id'] == 3969,'budget'] = 400
test.loc[test['id'] == 4196,'budget'] = 6
test.loc[test['id'] == 4221,'budget'] = 11
test.loc[test['id'] == 4222,'budget'] = 500
test.loc[test['id'] == 4285,'budget'] = 11
test.loc[test['id'] == 4319,'budget'] = 1
test.loc[test['id'] == 4639,'budget'] = 10
test.loc[test['id'] == 4719,'budget'] = 45
test.loc[test['id'] == 4822,'budget'] = 22
test.loc[test['id'] == 4829,'budget'] = 20
test.loc[test['id'] == 4969,'budget'] = 20
test.loc[test['id'] == 5021,'budget'] = 40 
test.loc[test['id'] == 5035,'budget'] = 1 
test.loc[test['id'] == 5063,'budget'] = 14 
test.loc[test['id'] == 5119,'budget'] = 2 
test.loc[test['id'] == 5214,'budget'] = 30 
test.loc[test['id'] == 5221,'budget'] = 50 
test.loc[test['id'] == 4903,'budget'] = 15
test.loc[test['id'] == 4983,'budget'] = 3
test.loc[test['id'] == 5102,'budget'] = 28
test.loc[test['id'] == 5217,'budget'] = 75
test.loc[test['id'] == 5224,'budget'] = 3 
test.loc[test['id'] == 5469,'budget'] = 20 
test.loc[test['id'] == 5840,'budget'] = 1 
test.loc[test['id'] == 5960,'budget'] = 30
test.loc[test['id'] == 6506,'budget'] = 11 
test.loc[test['id'] == 6553,'budget'] = 280
test.loc[test['id'] == 6561,'budget'] = 7
test.loc[test['id'] == 6582,'budget'] = 218
test.loc[test['id'] == 6638,'budget'] = 5
test.loc[test['id'] == 6749,'budget'] = 8 
test.loc[test['id'] == 6759,'budget'] = 50 
test.loc[test['id'] == 6856,'budget'] = 10
test.loc[test['id'] == 6858,'budget'] =  100
test.loc[test['id'] == 6876,'budget'] =  250
test.loc[test['id'] == 6972,'budget'] = 1
test.loc[test['id'] == 7079,'budget'] = 8000000
test.loc[test['id'] == 7150,'budget'] = 118
test.loc[test['id'] == 6506,'budget'] = 118
test.loc[test['id'] == 7225,'budget'] = 6
test.loc[test['id'] == 7231,'budget'] = 85
test.loc[test['id'] == 5222,'budget'] = 5
test.loc[test['id'] == 5322,'budget'] = 90
test.loc[test['id'] == 5350,'budget'] = 70
test.loc[test['id'] == 5378,'budget'] = 10
test.loc[test['id'] == 5545,'budget'] = 80
test.loc[test['id'] == 5810,'budget'] = 8
test.loc[test['id'] == 5926,'budget'] = 300
test.loc[test['id'] == 5927,'budget'] = 4
test.loc[test['id'] == 5986,'budget'] = 1
test.loc[test['id'] == 6053,'budget'] = 20
test.loc[test['id'] == 6104,'budget'] = 1
test.loc[test['id'] == 6130,'budget'] = 30
test.loc[test['id'] == 6301,'budget'] = 150
test.loc[test['id'] == 6276,'budget'] = 100
test.loc[test['id'] == 6473,'budget'] = 100
test.loc[test['id'] == 6842,'budget'] = 30

# Feature Engineering 

In [31]:
def convert_string_to_list(strVal):
    if type(strVal) is not str:
        return  []
    else:
        return ast.literal_eval(strVal)
    
def format_dict_column_and_extract_names(strVal, col="name"):
    listOfItems = convert_string_to_list(strVal)

    return list(map(lambda x: x[col], listOfItems))

def add_x_length_column(df, col):
    df[col + '_size'] = df[col].apply(lambda x: len(x))
    
    return df

def extract_genres(df):
    df['genres'] = df['genres'].apply(format_dict_column_and_extract_names)
    df = add_x_length_column(df, col='genres')
    temp = mlb.fit_transform(df.pop('genres'))
    temp_df = pd.DataFrame(temp, columns=list(map(lambda x: 'genres'+'_'+x,mlb.classes_)), 
                              )#index=temp.index)
    #print('extract_genres temp df shape: ', temp_df.shape)
    df = pd.merge(df, temp_df, how='inner', on=df.id)
    
    return df

def extract_common(df, col, limit, apply_dict_to_col=True):
    if apply_dict_to_col == True:
        df[col] = df[col].apply(format_dict_column_and_extract_names)
        
    companiesCount = df[col].apply(pd.Series).stack().value_counts().sort_values(axis=0, ascending=False)
    companiesToKeep = companiesCount[:limit].keys()
    
    add_x_length_column(df, col)
    df[col] = df[col].apply(lambda x: list(filter(lambda i: i in companiesToKeep, x)))
    
    temp = mlb.fit_transform(df.pop(col))
    temp_df = pd.DataFrame(temp, columns=list(map(lambda x: col+'_'+x,mlb.classes_)), 
                              )#index=temp.index)
    #print('extract_common temp df shape: ', temp_df.shape)
    #print('temp index: ',temp_df.index)
    df = pd.merge(df, temp_df, how='inner', left_index=True, right_index=True)

    return df

def last_year_metrics(df, agg_col='release_date_Year', calc_col=['revenue','imdb_id','popularity','budget'], leg=1):
    
    temp = df.groupby([agg_col]).agg(last_period_mean_revenue = (calc_col[0], 'mean'),
                                     last_period_median_revenue = (calc_col[0], 'median'),
                                     last_period_movies_count = (calc_col[1], 'count'),
                                     last_period_mean_popularity = (calc_col[2], 'mean'),
                                     last_period_median_popularity = (calc_col[2], 'median'),
                                     last_period_mean_budget = (calc_col[3], 'mean'),
                                     last_period_median_budget = (calc_col[3], 'median')).shift(leg).reset_index()

    return pd.merge(df, temp, on=['release_date_Year'], how='left')

def days_diff_from_last_movie(df, col='release_date'):#see if there is a need in buckts
    df = df.sort_values(by=[col])
    df['days_diff_from_last_movie'] = df[col].diff().fillna(0).dt.days
    
    return df

def ratios(df):
    df['budget_to_runtime'] = df['budget']/df['runtime']
    df['budget_to_cast_crew_size'] = df['budget']/(df['cast_size'] + df['crew_size'])
    df['budget_to_popularity'] = df['budget']/df['popularity']
    
    df['budget_to_year'] = df['budget'] / df['release_date_Year']
    df['budget_to_popularity_to_log_budget'] = df['budget_to_popularity'] / df['log_budget']
    df['year_to_log_budget'] = df['release_date_Year'] / df['log_budget']
    df['budget_to_runtime_to_year'] = df['budget_to_runtime'] / df['release_date_Year']
    
    return df

def add_datepart(df,col ='release_date'):
    df[col] = pd.to_datetime(df[col])
    df[col +'_Year']  = df[col].dt.year
    df[col +'_Month'] = df[col].dt.month
    df[col +'_day_of_month'] = df[col].dt.day
    df[col +'_week_day'] = df[col].dt.dayofweek
    df[col +'_is_quarter_start'] = df[col].dt.is_quarter_start
    df[col +'_is_quarter_start'] = df[col +'_is_quarter_start'].fillna(False)
    
    return df 

def preprocess(text):
    result = []
    for token in gensim.utils.simple_preprocess(text):
        if token not in gensim.parsing.preprocessing.STOPWORDS and len(token) > 3:
            result.append(token)
            
    return result

def extract_common_overview_words(df, col):
    df[col] = df[col].fillna('').astype(str).map(preprocess)
    df[col] = df[col].map(lambda x: list(set(x)))
    df = extract_common(df, col, limit=15, apply_dict_to_col=False)
    
    return df
  
def extract_gender(strVal, dict_key, dict_value):
    listOfItems = convert_string_to_list(strVal)

    Item = (list(filter(lambda lst: lst[dict_key] == dict_value, listOfItems)) or [None])[0]
    if type(Item) is dict:
        return Item['gender']
    else:
        return None
      
def add_gender(df, col, dict_key_in , dict_value_in):
    df[dict_key_in + '_' + str(dict_value_in) + '_gender'] = df[col].apply(extract_gender, args=[dict_key_in, dict_value_in])
    df[dict_key_in + '_' + str(dict_value_in) + '_gender'] = df[dict_key_in + '_' + str(dict_value_in) + '_gender'].fillna(2).astype('int64')
    
    return df

def run_main(train, test):

  test['train_test'] = 'test'  
  train['train_test'] = 'train'

  # work on full df
  print('working on full df..')
  df = pd.concat([train, test],axis=0, sort=True)
  df = extract_genres(df)
  df = extract_common(df, col='production_companies', limit=30)
  df = extract_common(df, col='production_countries', limit=20)
  df = extract_common(df, col='spoken_languages', limit=15)
  df = extract_common(df, col='Keywords', limit=15)
  
  df = add_gender(df, col='crew', dict_key_in='job', dict_value_in='Producer')
  df = add_gender(df, col='crew', dict_key_in='job', dict_value_in='Director')
  df = add_gender(df, col='cast', dict_key_in='order', dict_value_in=0)
  df = add_gender(df, col='cast', dict_key_in='order', dict_value_in=1)
  df = extract_common(df, col='cast', limit=15)
  df = extract_common(df, col='crew', limit=15)

  # fill missing values 1
  print('fill missing values 1...')
  df['cast_size'] = np.where(df['cast_size'] == 0, df['cast_size'].mean(), df['cast_size'])
  df['crew_size'] = np.where(df['crew_size'] == 0, df['crew_size'].mean(), df['crew_size'])
  
  # add datepart
  print('add datepart...')
  df = add_datepart(df,col = 'release_date') 

  # split for data leak prevention
  print('split for data leak prevention and run main...')
  train = df[df['train_test'] == 'train']
  test = df[df['train_test'] == 'test']

  train = last_year_metrics(train)
  train = days_diff_from_last_movie(train)
  train['revenue'] = np.where(train['revenue'] == 0, train['last_period_median_revenue'], train['revenue'])
  train['revenue'] = np.where(train['revenue'] == 0, train["revenue"].mean(), train['revenue'])
  train = extract_common_overview_words(train, col='overview')

  test = last_year_metrics(test)
  test = days_diff_from_last_movie(test)
  test['revenue'] = np.where(test['revenue'] == 0, test['last_period_median_revenue'], test['revenue'])
  test['revenue'] = np.where(test['revenue'] == 0, test["revenue"].mean(), test['revenue'])
  test = extract_common_overview_words(test, col='overview')

  # addtion by https://www.kaggle.com/artgor/eda-feature-engineering-and-model-interpretation
  train['log_budget'] = np.log1p(train['budget'])
  test['log_budget'] = np.log1p(test['budget'])
  train['has_homepage'] = 0
  train.loc[train['homepage'].isnull() == False, 'has_homepage'] = 1
  test['has_homepage'] = 0
  test.loc[test['homepage'].isnull() == False, 'has_homepage'] = 1

  df = pd.concat([train, test],axis=0, sort=True)

  # fill missing values 2
  print('fill missing values 2...')
  df['budget'] = np.where(df['budget'] == 0, df['last_period_median_budget'], df['budget'])
  df['budget'] = np.where(df['budget'] == 0, df["budget"].mean(), df['budget'])
  df['production_companies_size'] = np.where(df['production_companies_size'] == 0, df["production_companies_size"].mean(), df['production_companies_size'])
  df['production_countries_size'] = np.where(df['production_countries_size'] == 0, df["production_countries_size"].mean(), df['production_countries_size'])
  df['Keywords_size'] = np.where(df['Keywords_size'] == 0, df["Keywords_size"].mean(), df['Keywords_size'])

  # run ratios
  print('run ratios...')
  df = ratios(df)

  # fill missing values 3
  print('fill missing values 3...')
  df = df.replace([np.inf, -np.inf], np.nan)
  df['budget_to_runtime'] = np.where(df['budget_to_runtime'] == np.nan, df["budget_to_runtime"].mean(), df['budget_to_runtime'])
  df['belongs_to_collection_bool'] = np.where(df['belongs_to_collection'].isna(), 0, 1)
  df.drop(['belongs_to_collection'], axis=1, inplace=True)

  # counting the numbers of words in the movie title 
  print('counting the numbers of words in the movie title..')
  df['original_title_words'] = df.original_title.str.split()
  df['original_title_words'] = df['original_title_words'].apply(lambda x: len(x))

  df = pd.get_dummies(df, prefix=['job_Producer_gender', 'job_Director_gender', 'order_0_gender', 'order_1_gender'], 
                          columns=['job_Producer_gender', 'job_Director_gender', 'order_0_gender', 'order_1_gender'])

  # removing highly correlated features
  print('removing highly correlated features...')
  df.drop(["crew_Bob Weinstein", "release_date_Year"], inplace=True, axis=1)

  # normalize features
  scaler = preprocessing.MinMaxScaler()
  df["budget_to_popularity"] = scaler.fit_transform(df[["budget_to_popularity"]])
  

  return df

df = run_main(train, test)
df.head()

working on full df..
fill missing values 1...
add datepart...
split for data leak prevention and run main...


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:61: FutureWarning: Passing integers to fillna is deprecated, will raise a TypeError in a future version.  To retain the old behavior, pass pd.Timedelta(seconds=n) instead.


fill missing values 2...
run ratios...
fill missing values 3...
counting the numbers of words in the movie title..
removing highly correlated features...


,Keywords_aftercreditsstinger,Keywords_based on novel,Keywords_biography,Keywords_duringcreditsstinger,Keywords_dystopia,Keywords_friendship,Keywords_independent film,Keywords_love,Keywords_murder,Keywords_police,Keywords_revenge,Keywords_sex,Keywords_size,Keywords_sport,Keywords_violence,Keywords_woman director,budget,cast_Bruce Willis,cast_Christopher Walken,cast_J.K. Simmons,cast_John Goodman,cast_Julianne Moore,cast_Liam Neeson,cast_Matt Damon,cast_Morgan Freeman,cast_Nicolas Cage,cast_Robert De Niro,cast_Robin Williams,cast_Samuel L. Jackson,cast_Steve Buscemi,cast_Sylvester Stallone,cast_Willem Dafoe,cast_size,catfinal,crew_Avy Kaufman,crew_Deborah Aquila,crew_Francine Maisler,crew_Hans Zimmer,crew_Harvey Weinstein,crew_James Horner,crew_James Newton Howard,crew_Jerry Goldsmith,crew_Kerry Barden,crew_Luc Besson,crew_Mary Vernieu,crew_Robert Rodriguez,crew_Steven Spielberg,crew_Tricia Wood,crew_size,days_diff_from_last_movie,genres_Action,genres_Adventure,genres_Animation,genres_Comedy,genres_Crime,genres_Documentary,genres_Drama,genres_Family,genres_Fantasy,genres_Foreign,genres_History,genres_Horror,genres_Music,genres_Mystery,genres_Romance,genres_Science Fiction,genres_TV Movie,genres_Thriller,genres_War,genres_Western,genres_size,has_homepage,homepage,id,imdb_id,key_0,last_period_mean_budget,last_period_mean_popularity,last_period_mean_revenue,last_period_median_budget,last_period_median_popularity,last_period_median_revenue,last_period_movies_count,log_budget,original_language,original_title,overview_family,overview_father,overview_film,overview_finds,overview_friends,overview_help,overview_life,overview_love,overview_size,overview_story,overview_time,overview_wife,overview_woman,overview_world,overview_year,overview_years,overview_young,popularity,poster_path,production_companies_Amblin Entertainment,production_companies_BBC Films,production_companies_Canal+,production_companies_Columbia Pictures,production_companies_Columbia Pictures Corporation,production_companies_Dimension Films,production_companies_DreamWorks SKG,production_companies_Dune Entertainment,production_companies_Fox 2000 Pictures,production_companies_Fox Searchlight Pictures,production_companies_Hollywood Pictures,production_companies_Lionsgate,production_companies_Metro-Goldwyn-Mayer (MGM),production_companies_Miramax Films,production_companies_New Line Cinema,production_companies_Orion Pictures,production_companies_Paramount Pictures,production_companies_Regency Enterprises,production_companies_Relativity Media,production_companies_StudioCanal,production_companies_Summit Entertainment,production_companies_Touchstone Pictures,production_companies_TriStar Pictures,production_companies_Twentieth Century Fox Film Corporation,production_companies_United Artists,production_companies_Universal Pictures,production_companies_Village Roadshow Pictures,production_companies_Walt Disney Pictures,production_companies_Warner Bros.,production_companies_Working Title Films,production_companies_size,production_countries_Australia,production_countries_Belgium,production_countries_Canada,production_countries_China,production_countries_Denmark,production_countries_France,production_countries_Germany,production_countries_Hong Kong,production_countries_India,production_countries_Ireland,production_countries_Italy,production_countries_Japan,production_countries_Mexico,production_countries_Netherlands,production_countries_Russia,production_countries_South Korea,production_countries_Spain,production_countries_Sweden,production_countries_United Kingdom,production_countries_United States of America,production_countries_size,release_date,release_date_Month,release_date_day_of_month,release_date_is_quarter_start,release_date_week_day,revenue,runtime,spoken_languages_,spoken_languages_Deutsch,spoken_languages_English,spoken_languages_Español,spoken_languages_Français,spoken_languages_Italiano,spoken_languages_Polski,spoken_languages_Português,spoken_languages_Pусский,spoken_lan

In [0]:
#import pandas_profiling
#report = pandas_profiling.ProfileReport(df)
#report.to_file("your_report.html")

# Modeling

In [0]:
model_df = df.copy()
# model_df['id'] =  model_df.index
model_df = model_df.drop(["homepage", "imdb_id", "original_language", "original_title", "poster_path", "release_date", "status", "tagline", "title"], axis=1)

train = model_df[model_df['train_test'] == 'train']
test = model_df[model_df['train_test'] == 'test']

#train_y = train[['revenue']]
#test_y = test[['revenue']]
train = train.drop(['train_test'],axis=1)
test = test.drop(['train_test'] ,axis=1)

# scale_train = StandardScaler().fit(train)
# scale_df = scale_train.transform(train)
# train = pd.DataFrame(scale_df,columns=train.columns)#.merge(train_y, left_index=True, right_index=True)

# scale_test = StandardScaler().fit(test)
# scale_df = scale_test.transform(test)
# test = pd.DataFrame(scale_df,columns=test.columns)#.merge(test_y, left_index=True, right_index=True)


def score(data, y):
    validation_res = pd.DataFrame(
    {"id": data["id"].values,
     "transactionrevenue": data["revenue"].values,
     "predictedrevenue": np.expm1(y)}) 

    print(validation_res.head())

    validation_res = validation_res.groupby("id")["transactionrevenue", "predictedrevenue"].sum().reset_index()
    
    print(validation_res.head())
    return  np.sqrt(mean_squared_error((np.log1p(validation_res["transactionrevenue"].values)), #np.log1p
                                     (np.log1p(validation_res["predictedrevenue"].values)))) #np.log1p
    
class KFoldValidation():
    def __init__(self, data, n_splits=5):
        unique_vis = np.array(sorted(data['id'].astype(str).unique()))
        folds = GroupKFold(n_splits)
        ids = np.arange(data.shape[0])
        
        self.fold_ids = []
        for trn_vis, val_vis in folds.split(X=unique_vis, y=unique_vis, groups=unique_vis):
            self.fold_ids.append([
                    ids[data['id'].astype(str).isin(unique_vis[trn_vis])],
                    ids[data['id'].astype(str).isin(unique_vis[val_vis])]
                ])
            
    def validate(self, train, test, features, model, name="", prepare_stacking=False, 
                 fit_params={"early_stopping_rounds": 500, "verbose": 100, "eval_metric": "rmse"}):
        model.FI = pd.DataFrame(index=features)
        full_score = 0
        
        if prepare_stacking:
            test[name] = 0
            train[name] = np.NaN
        
        for fold_id, (trn, val) in enumerate(self.fold_ids):
            devel = train[features].iloc[trn]
            y_devel = np.log1p(train["revenue"].iloc[trn])
            valid = train[features].iloc[val]
            y_valid = np.log1p(train["revenue"].iloc[val])
                       
            print("Fold ", fold_id, ":")
            model.fit(devel, y_devel, eval_set=[(valid, y_valid)], **fit_params)
            
            if len(model.feature_importances_) == len(features):  # some bugs in catboost?
                model.FI['fold' + str(fold_id)] = model.feature_importances_ / model.feature_importances_.sum()

            predictions = model.predict(valid)
            predictions[predictions < 0] = 0
            print("Fold ", fold_id, " error: ", mean_squared_error(y_valid, predictions)**0.5)
            
            fold_score = score(train.iloc[val], predictions)
            full_score += fold_score / len(self.fold_ids)
            print("Fold ", fold_id, " score: ", fold_score)
            
            if prepare_stacking:
                train[name].iloc[val] = np.expm1(predictions)
                
                test_predictions = model.predict(test[features])
                print(test_predictions)
                test_predictions[test_predictions < 0] = 0
                test[name] += np.expm1(test_predictions) / len(self.fold_ids)
                
        print("Final rmsle score: ", full_score)
        return full_score


## Models config

In [0]:
lgbmodel = lgb.LGBMRegressor(n_estimators=20000, 
                             objective='regression', 
                             metric='rmse',
                             max_depth = 25,
                             num_leaves=10, 
                             min_child_samples=100,
                             learning_rate=0.001,
                             boosting = 'gbdt',
                             min_data_in_leaf= 10,
                             feature_fraction = 0.2,
                             bagging_freq = 1,
                             bagging_fraction = 0.9,
                             importance_type='gain',
                             lambda_l1 = 0.2,
                             bagging_seed=42, 
                             subsample=.8, 
                             colsample_bytree=.9,
                             use_best_model=True)

xgbmodel = xgb.XGBRegressor(max_depth=10, 
                            learning_rate=0.01, 
                            n_estimators=20000, 
                            objective='reg:linear', 
                            gamma=1.45, 
                            seed=42, 
                            silent=True,
                            subsample=0.7, 
                            colsample_bytree=0.8, 
                            colsample_bylevel=0.50)

catmodel = cat.CatBoostRegressor(iterations=20000, 
                                 learning_rate=0.01, 
                                 depth=6,
                                 loss_function = "RMSE",
                                 boost_from_average = True,
                                 colsample_bylevel=0.8,
                                 bagging_temperature = 0.2,
                                 metric_period = None,
                                 random_seed=2345,
                                 l2_leaf_reg = 0.2)

In [0]:
Kfolder = KFoldValidation(train)

In [35]:
Kfolder.validate(train, test, train.columns.drop('revenue') , lgbmodel, name="lgbfinal", prepare_stacking=True) 

Fold  0 :
Training until validation scores don't improve for 500 rounds.
[100]	valid_0's rmse: 3.05079
[200]	valid_0's rmse: 2.95802
[300]	valid_0's rmse: 2.87816
[400]	valid_0's rmse: 2.81161
[500]	valid_0's rmse: 2.74798
[600]	valid_0's rmse: 2.6926
[700]	valid_0's rmse: 2.64342
[800]	valid_0's rmse: 2.59802
[900]	valid_0's rmse: 2.55931
[1000]	valid_0's rmse: 2.52427
[1100]	valid_0's rmse: 2.49489
[1200]	valid_0's rmse: 2.4677
[1300]	valid_0's rmse: 2.44388
[1400]	valid_0's rmse: 2.42244
[1500]	valid_0's rmse: 2.40295
[1600]	valid_0's rmse: 2.38517
[1700]	valid_0's rmse: 2.3693
[1800]	valid_0's rmse: 2.35531
[1900]	valid_0's rmse: 2.34294
[2000]	valid_0's rmse: 2.33126
[2100]	valid_0's rmse: 2.32101
[2200]	valid_0's rmse: 2.3112
[2300]	valid_0's rmse: 2.30199
[2400]	valid_0's rmse: 2.29429
[2500]	valid_0's rmse: 2.28674
[2600]	valid_0's rmse: 2.2797
[2700]	valid_0's rmse: 2.27349
[2800]	valid_0's rmse: 2.26781
[2900]	valid_0's rmse: 2.26258
[3000]	valid_0's rmse: 2.25759
[3100]	vali

/usr/local/lib/python3.6/dist-packages/pandas/core/indexing.py:205: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_with_indexer(indexer, value)


[12.80098009 12.4745098   5.33631345 ... 14.32135304 13.41230551
  8.59470688]
Fold  1 :
Training until validation scores don't improve for 500 rounds.
[100]	valid_0's rmse: 2.71755
[200]	valid_0's rmse: 2.62785
[300]	valid_0's rmse: 2.5378
[400]	valid_0's rmse: 2.46101
[500]	valid_0's rmse: 2.39256
[600]	valid_0's rmse: 2.33359
[700]	valid_0's rmse: 2.27907
[800]	valid_0's rmse: 2.23172
[900]	valid_0's rmse: 2.18837
[1000]	valid_0's rmse: 2.15147
[1100]	valid_0's rmse: 2.11883
[1200]	valid_0's rmse: 2.09079
[1300]	valid_0's rmse: 2.06486
[1400]	valid_0's rmse: 2.04152
[1500]	valid_0's rmse: 2.01966
[1600]	valid_0's rmse: 2.00078
[1700]	valid_0's rmse: 1.98386
[1800]	valid_0's rmse: 1.96879
[1900]	valid_0's rmse: 1.95503
[2000]	valid_0's rmse: 1.942
[2100]	valid_0's rmse: 1.93044
[2200]	valid_0's rmse: 1.92032
[2300]	valid_0's rmse: 1.91054
[2400]	valid_0's rmse: 1.90212
[2500]	valid_0's rmse: 1.89412
[2600]	valid_0's rmse: 1.88742
[2700]	valid_0's rmse: 1.88092
[2800]	valid_0's rmse: 

/usr/local/lib/python3.6/dist-packages/pandas/core/indexing.py:205: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_with_indexer(indexer, value)


[13.35906949 11.92272391  2.32019826 ... 14.33299997 13.19460873
 10.9145629 ]
Fold  2 :
Training until validation scores don't improve for 500 rounds.
[100]	valid_0's rmse: 2.89685
[200]	valid_0's rmse: 2.78762
[300]	valid_0's rmse: 2.67964
[400]	valid_0's rmse: 2.58624
[500]	valid_0's rmse: 2.5004
[600]	valid_0's rmse: 2.42609
[700]	valid_0's rmse: 2.35841
[800]	valid_0's rmse: 2.29707
[900]	valid_0's rmse: 2.24038
[1000]	valid_0's rmse: 2.192
[1100]	valid_0's rmse: 2.14999
[1200]	valid_0's rmse: 2.11405
[1300]	valid_0's rmse: 2.07965
[1400]	valid_0's rmse: 2.04895
[1500]	valid_0's rmse: 2.02075
[1600]	valid_0's rmse: 1.99525
[1700]	valid_0's rmse: 1.97288
[1800]	valid_0's rmse: 1.9527
[1900]	valid_0's rmse: 1.93469
[2000]	valid_0's rmse: 1.91741
[2100]	valid_0's rmse: 1.90188
[2200]	valid_0's rmse: 1.88819
[2300]	valid_0's rmse: 1.87542
[2400]	valid_0's rmse: 1.86364
[2500]	valid_0's rmse: 1.85341
[2600]	valid_0's rmse: 1.8443
[2700]	valid_0's rmse: 1.83558
[2800]	valid_0's rmse: 1.

...

/usr/local/lib/python3.6/dist-packages/pandas/core/indexing.py:205: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_with_indexer(indexer, value)


[12.95504103 12.80955188  2.77834994 ... 14.41699113 13.16265879
 10.24705688]
Fold  4 :
Training until validation scores don't improve for 500 rounds.
[100]	valid_0's rmse: 3.07747
[200]	valid_0's rmse: 2.97716
[300]	valid_0's rmse: 2.8896
[400]	valid_0's rmse: 2.81182
[500]	valid_0's rmse: 2.74236
[600]	valid_0's rmse: 2.67969
[700]	valid_0's rmse: 2.62127
[800]	valid_0's rmse: 2.57083
[900]	valid_0's rmse: 2.5297
[1000]	valid_0's rmse: 2.49229
[1100]	valid_0's rmse: 2.45768
[1200]	valid_0's rmse: 2.42765
[1300]	valid_0's rmse: 2.40051
[1400]	valid_0's rmse: 2.3771
[1500]	valid_0's rmse: 2.35402
[1600]	valid_0's rmse: 2.33575
[1700]	valid_0's rmse: 2.31717
[1800]	valid_0's rmse: 2.3015
[1900]	valid_0's rmse: 2.28645
[2000]	valid_0's rmse: 2.27305
[2100]	valid_0's rmse: 2.2612
[2200]	valid_0's rmse: 2.25134
[2300]	valid_0's rmse: 2.24216
[2400]	valid_0's rmse: 2.23368
[2500]	valid_0's rmse: 2.22551
[2600]	valid_0's rmse: 2.21819
[2700]	valid_0's rmse: 2.21188
[2800]	valid_0's rmse: 2.

/usr/local/lib/python3.6/dist-packages/pandas/core/indexing.py:205: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_with_indexer(indexer, value)


[12.95300806 12.58959504  1.54096664 ... 14.67431405 13.42644263
  7.91277484]
Final rmsle score:  1.8569874385762821


1.8569874385762821

In [36]:
Kfolder.validate(train, test, train.columns.drop('revenue'), xgbmodel, name="xgbfinal", prepare_stacking=True)

Fold  0 :
[0]	validation_0-rmse:15.5546
Will train until validation_0-rmse hasn't improved in 500 rounds.


/usr/local/lib/python3.6/dist-packages/xgboost/core.py:587: FutureWarning: Series.base is deprecated and will be removed in a future version
  if getattr(data, 'base', None) is not None and \


[100]	validation_0-rmse:6.01617
[200]	validation_0-rmse:3.01239
[300]	validation_0-rmse:2.36457
[400]	validation_0-rmse:2.27545
[500]	validation_0-rmse:2.26923
[600]	validation_0-rmse:2.27181
[700]	validation_0-rmse:2.2749
[800]	validation_0-rmse:2.27804
[900]	validation_0-rmse:2.27945
[1000]	validation_0-rmse:2.28103
Stopping. Best iteration:
[516]	validation_0-rmse:2.26895

Fold  0  error:  2.2689511313483712
     id  transactionrevenue  predictedrevenue
0  1764          2500000.00         516729.75
1  1162              121.00          88473.58
2   556           600000.00         146487.33
3  2008          2610000.00        3364372.50
4   993          3202000.00        1728671.75
   id  transactionrevenue  predictedrevenue
0  20         85167639.00       75524160.00
1  22         36690067.00       32519916.00
2  27         13835130.00        1383311.75
3  41           163000.00        1605366.38
4  47        660940780.00      274267136.00
Fold  0  score:  2.2689511313483712


/usr/local/lib/python3.6/dist-packages/pandas/core/indexing.py:205: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_with_indexer(indexer, value)


[11.920015  12.240438   3.3561385 ... 13.802613  12.359708   9.096441 ]
Fold  1 :
[0]	validation_0-rmse:15.5639
Will train until validation_0-rmse hasn't improved in 500 rounds.


...

/usr/local/lib/python3.6/dist-packages/xgboost/core.py:587: FutureWarning: Series.base is deprecated and will be removed in a future version
  if getattr(data, 'base', None) is not None and \


[100]	validation_0-rmse:6.06828
[200]	validation_0-rmse:2.9891
[300]	validation_0-rmse:2.2793
[400]	validation_0-rmse:2.1742
[500]	validation_0-rmse:2.16855
[600]	validation_0-rmse:2.17256
[700]	validation_0-rmse:2.17658
[800]	validation_0-rmse:2.17715
[900]	validation_0-rmse:2.17873
Stopping. Best iteration:
[454]	validation_0-rmse:2.16776

Fold  4  error:  2.1677562282203717
     id  transactionrevenue  predictedrevenue
0  2681           966878.00        1068973.88
1  2578               97.00          15315.37
2   845          1012189.00        2286897.00
3   302          2850000.00         333066.91
4   822          2600000.00        6429453.00
   id  transactionrevenue  predictedrevenue
0   1         12314651.00       36043396.00
1   3         13092000.00       19578036.00
2   7         85446075.00       15133899.00
3  42         16217773.00       20220710.00
4  43         24362772.00        9849995.00
Fold  4  score:  2.1677562282203717


/usr/local/lib/python3.6/dist-packages/pandas/core/indexing.py:205: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_with_indexer(indexer, value)


[12.124469  11.322226   3.3152816 ... 14.300222  12.597043   7.013679 ]
Final rmsle score:  1.956136938685833


1.956136938685833

In [40]:
Kfolder.validate(train, test, train.columns.drop('revenue') , catmodel, name="catfinal", prepare_stacking=True,
               fit_params={"use_best_model": True, "verbose": 100})

Fold  0 :
0:	learn: 2.9909886	test: 3.1398171	best: 3.1398171 (0)	total: 6.19ms	remaining: 2m 3s
100:	learn: 2.0586720	test: 2.3934705	best: 2.3934705 (100)	total: 493ms	remaining: 1m 37s
200:	learn: 1.8092936	test: 2.2306855	best: 2.2306855 (200)	total: 983ms	remaining: 1m 36s
300:	learn: 1.7123179	test: 2.1975681	best: 2.1975681 (300)	total: 1.46s	remaining: 1m 35s
400:	learn: 1.6555734	test: 2.1881071	best: 2.1876787 (397)	total: 1.95s	remaining: 1m 35s
500:	learn: 1.6051030	test: 2.1844561	best: 2.1840272 (492)	total: 2.44s	remaining: 1m 35s
600:	learn: 1.5612942	test: 2.1847074	best: 2.1840272 (492)	total: 2.92s	remaining: 1m 34s
700:	learn: 1.5175877	test: 2.1868935	best: 2.1840272 (492)	total: 3.4s	remaining: 1m 33s
800:	learn: 1.4733667	test: 2.1897072	best: 2.1840272 (492)	total: 3.87s	remaining: 1m 32s
900:	learn: 1.4317062	test: 2.1901800	best: 2.1840272 (492)	total: 4.34s	remaining: 1m 31s
1000:	learn: 1.3835927	test: 2.1926770	best: 2.1840272 (492)	total: 4.81s	remaining: 

/usr/local/lib/python3.6/dist-packages/pandas/core/indexing.py:205: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_with_indexer(indexer, value)


0:	learn: 3.0699484	test: 2.8125305	best: 2.8125305 (0)	total: 4.75ms	remaining: 1m 35s
100:	learn: 2.1602943	test: 2.0024027	best: 2.0024027 (100)	total: 480ms	remaining: 1m 34s
200:	learn: 1.9156841	test: 1.8235185	best: 1.8235185 (200)	total: 951ms	remaining: 1m 33s
300:	learn: 1.8192544	test: 1.7833840	best: 1.7833501 (299)	total: 1.44s	remaining: 1m 34s
400:	learn: 1.7560246	test: 1.7776847	best: 1.7773585 (396)	total: 1.9s	remaining: 1m 32s
500:	learn: 1.7029909	test: 1.7759007	best: 1.7751524 (489)	total: 2.38s	remaining: 1m 32s
600:	learn: 1.6600685	test: 1.7755549	best: 1.7751524 (489)	total: 2.87s	remaining: 1m 32s
700:	learn: 1.6144639	test: 1.7752666	best: 1.7749475 (696)	total: 3.35s	remaining: 1m 32s
800:	learn: 1.5635719	test: 1.7754316	best: 1.7740226 (733)	total: 3.83s	remaining: 1m 31s
900:	learn: 1.5104921	test: 1.7772704	best: 1.7740226 (733)	total: 4.3s	remaining: 1m 31s
1000:	learn: 1.4591582	test: 1.7797848	best: 1.7740226 (733)	total: 4.78s	remaining: 1m 30s
110

/usr/local/lib/python3.6/dist-packages/pandas/core/indexing.py:205: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_with_indexer(indexer, value)


0:	learn: 3.0213218	test: 3.0082686	best: 3.0082686 (0)	total: 5.27ms	remaining: 1m 45s
100:	learn: 2.1563746	test: 2.0308246	best: 2.0308246 (100)	total: 556ms	remaining: 1m 49s
200:	learn: 1.9225746	test: 1.7820091	best: 1.7820091 (200)	total: 1.06s	remaining: 1m 44s
300:	learn: 1.8309654	test: 1.7170206	best: 1.7170206 (300)	total: 1.57s	remaining: 1m 42s
400:	learn: 1.7673833	test: 1.6985789	best: 1.6985789 (400)	total: 2.08s	remaining: 1m 41s
500:	learn: 1.7128718	test: 1.6956972	best: 1.6951602 (481)	total: 2.61s	remaining: 1m 41s
600:	learn: 1.6675225	test: 1.6933272	best: 1.6930151 (595)	total: 3.14s	remaining: 1m 41s
700:	learn: 1.6202141	test: 1.6919357	best: 1.6916318 (674)	total: 3.63s	remaining: 1m 39s
800:	learn: 1.5725883	test: 1.6932033	best: 1.6916318 (674)	total: 4.14s	remaining: 1m 39s
900:	learn: 1.5219182	test: 1.6944585	best: 1.6916318 (674)	total: 4.66s	remaining: 1m 38s
1000:	learn: 1.4714017	test: 1.6927805	best: 1.6916318 (674)	total: 5.16s	remaining: 1m 37s
1

...

0:	learn: 3.0364091	test: 2.9512816	best: 2.9512816 (0)	total: 4.99ms	remaining: 1m 39s
100:	learn: 2.1687864	test: 1.9700150	best: 1.9700150 (100)	total: 472ms	remaining: 1m 33s
200:	learn: 1.9300028	test: 1.7394011	best: 1.7394011 (200)	total: 963ms	remaining: 1m 34s
300:	learn: 1.8347196	test: 1.6948283	best: 1.6945107 (292)	total: 1.47s	remaining: 1m 35s
400:	learn: 1.7718523	test: 1.6828966	best: 1.6824869 (393)	total: 1.95s	remaining: 1m 35s
500:	learn: 1.7181382	test: 1.6838515	best: 1.6823870 (410)	total: 2.44s	remaining: 1m 34s
600:	learn: 1.6726090	test: 1.6871857	best: 1.6823870 (410)	total: 2.92s	remaining: 1m 34s
700:	learn: 1.6213162	test: 1.6913823	best: 1.6823870 (410)	total: 3.41s	remaining: 1m 33s
800:	learn: 1.5735563	test: 1.6945799	best: 1.6823870 (410)	total: 3.87s	remaining: 1m 32s
900:	learn: 1.5206897	test: 1.6972728	best: 1.6823870 (410)	total: 4.34s	remaining: 1m 31s
1000:	learn: 1.4690195	test: 1.7017055	best: 1.6823870 (410)	total: 4.84s	remaining: 1m 31s
1

/usr/local/lib/python3.6/dist-packages/pandas/core/indexing.py:205: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_with_indexer(indexer, value)


0:	learn: 2.9777618	test: 3.1834337	best: 3.1834337 (0)	total: 4.82ms	remaining: 1m 36s
100:	learn: 2.0736767	test: 2.6530872	best: 2.6530872 (100)	total: 491ms	remaining: 1m 36s
200:	learn: 1.8344440	test: 2.5635112	best: 2.5551224 (180)	total: 962ms	remaining: 1m 34s
300:	learn: 1.7448886	test: 2.5317349	best: 2.5312843 (295)	total: 1.43s	remaining: 1m 33s
400:	learn: 1.6877693	test: 2.5134834	best: 2.5134834 (400)	total: 1.92s	remaining: 1m 34s
500:	learn: 1.6398011	test: 2.5064411	best: 2.5016649 (487)	total: 2.41s	remaining: 1m 33s
600:	learn: 1.5943800	test: 2.5230603	best: 2.5016649 (487)	total: 2.89s	remaining: 1m 33s
700:	learn: 1.5522134	test: 2.5339174	best: 2.5016649 (487)	total: 3.36s	remaining: 1m 32s
800:	learn: 1.5096806	test: 2.5285153	best: 2.5016649 (487)	total: 3.85s	remaining: 1m 32s
900:	learn: 1.4612869	test: 2.5252860	best: 2.5016649 (487)	total: 4.34s	remaining: 1m 31s
1000:	learn: 1.4155628	test: 2.4993926	best: 2.4993248 (999)	total: 4.84s	remaining: 1m 31s
1

/usr/local/lib/python3.6/dist-packages/pandas/core/indexing.py:205: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_with_indexer(indexer, value)


1.9240176808468894

# Model Stacking

In [18]:
train.head(5)

,Keywords_aftercreditsstinger,Keywords_based on novel,Keywords_biography,Keywords_duringcreditsstinger,Keywords_dystopia,Keywords_friendship,Keywords_independent film,Keywords_love,Keywords_murder,Keywords_police,Keywords_revenge,Keywords_sex,Keywords_size,Keywords_sport,Keywords_violence,Keywords_woman director,budget,cast_Bruce Willis,cast_Christopher Walken,cast_J.K. Simmons,cast_John Goodman,cast_Julianne Moore,cast_Liam Neeson,cast_Matt Damon,cast_Morgan Freeman,cast_Nicolas Cage,cast_Robert De Niro,cast_Robin Williams,cast_Samuel L. Jackson,cast_Steve Buscemi,cast_Sylvester Stallone,cast_Willem Dafoe,cast_size,crew_Avy Kaufman,crew_Deborah Aquila,crew_Francine Maisler,crew_Hans Zimmer,crew_Harvey Weinstein,crew_James Horner,crew_James Newton Howard,crew_Jerry Goldsmith,crew_Kerry Barden,crew_Luc Besson,crew_Mary Vernieu,crew_Robert Rodriguez,crew_Steven Spielberg,crew_Tricia Wood,crew_size,days_diff_from_last_movie,genres_Action,genres_Adventure,genres_Animation,genres_Comedy,genres_Crime,genres_Documentary,genres_Drama,genres_Family,genres_Fantasy,genres_Foreign,genres_History,genres_Horror,genres_Music,genres_Mystery,genres_Romance,genres_Science Fiction,genres_TV Movie,genres_Thriller,genres_War,genres_Western,genres_size,has_homepage,id,key_0,last_period_mean_budget,last_period_mean_popularity,last_period_mean_revenue,last_period_median_budget,last_period_median_popularity,last_period_median_revenue,last_period_movies_count,log_budget,overview_family,overview_father,overview_film,overview_finds,overview_friends,overview_help,overview_life,overview_love,overview_size,overview_story,overview_time,overview_wife,overview_woman,overview_world,overview_year,overview_years,overview_young,popularity,production_companies_Amblin Entertainment,production_companies_BBC Films,production_companies_Canal+,production_companies_Columbia Pictures,production_companies_Columbia Pictures Corporation,production_companies_Dimension Films,production_companies_DreamWorks SKG,production_companies_Dune Entertainment,production_companies_Fox 2000 Pictures,production_companies_Fox Searchlight Pictures,production_companies_Hollywood Pictures,production_companies_Lionsgate,production_companies_Metro-Goldwyn-Mayer (MGM),production_companies_Miramax Films,production_companies_New Line Cinema,production_companies_Orion Pictures,production_companies_Paramount Pictures,production_companies_Regency Enterprises,production_companies_Relativity Media,production_companies_StudioCanal,production_companies_Summit Entertainment,production_companies_Touchstone Pictures,production_companies_TriStar Pictures,production_companies_Twentieth Century Fox Film Corporation,production_companies_United Artists,production_companies_Universal Pictures,production_companies_Village Roadshow Pictures,production_companies_Walt Disney Pictures,production_companies_Warner Bros.,production_companies_Working Title Films,production_companies_size,production_countries_Australia,production_countries_Belgium,production_countries_Canada,production_countries_China,production_countries_Denmark,production_countries_France,production_countries_Germany,production_countries_Hong Kong,production_countries_India,production_countries_Ireland,production_countries_Italy,production_countries_Japan,production_countries_Mexico,production_countries_Netherlands,production_countries_Russia,production_countries_South Korea,production_countries_Spain,production_countries_Sweden,production_countries_United Kingdom,production_countries_United States of America,production_countries_size,release_date_Month,release_date_day_of_month,release_date_is_quarter_start,release_date_week_day,revenue,runtime,spoken_languages_,spoken_languages_Deutsch,spoken_languages_English,spoken_languages_Español,spoken_languages_Français,spoken_languages_Italiano,spoken_languages_Polski,spoken_languages_Português,spoken_languages_Pусский,spoken_languages_size,spoken_languages_العربية,spoken_languages_हिन्दी,spoken_languages_广州话 / 

In [41]:
train['PredictedLogRevenue'] = 0.4 * train["lgbfinal"] + \
                               0.2 * train["xgbfinal"] + \
                               0.4 * train["catfinal"]



np.sqrt(mean_squared_error((np.log1p(train["revenue"].values)), #np.log1p
                                     (np.log1p(train["lgbfinal"].values)))) #np.log1p

1.8683246755162224

In [0]:
test.head(12)

In [0]:
submission = test[['id','PredictedLogRevenue']]
submission.to_csv('sub.csv')

# Charts 

In [0]:
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings('ignore')
%matplotlib inline

plt.figure(figsize=(20,7))
lgb.plot_importance(lgbmodel,figsize=(10,40))
plt.yticks(fontsize=20)
plt.show()